# COVID19 - Brazil States

# import Ray

In [1]:
from environs import Env
env = Env()
env.str("CUDA_DEVICE_ORDER",'PCI_BUS_ID')
env.str("CUDA_VISIBLE_DEVICES",'0,1,2,3,4')
env.int("NUMBA_ENABLE_CUDASIM",1)
env.bool("OMPI_MCA_opal_cuda_support",True)

import os
import ray
ray.shutdown()
ray.init()

@ray.remote(num_gpus=1)
def use_gpu():
    print("ray.get_gpu_ids(): {}".format(ray.get_gpu_ids()[0]))
    print("CUDA_VISIBLE_DEVICES: {}".format(os.environ["CUDA_VISIBLE_DEVICES"]))

use_gpu.remote()

2021-03-03 08:59:05,331	INFO services.py:1092 -- View the Ray dashboard at http://127.0.0.1:8266


ObjectRef(df5a1a828c9685d3ffffffff0100000001000000)

# Import Modules

In [2]:
import urllib.request
import pandas as pd
import numpy as np
from datetime import datetime,timedelta

(pid=1253) ray.get_gpu_ids(): 0
(pid=1253) CUDA_VISIBLE_DEVICES: 0


# Download data

In [3]:
import get_data_v2
LoadData=False

if LoadData:
    get_data.get_data()

In [4]:
df = pd.read_csv("data/dados_total_estados.csv",compression='gzip')
df.drop('index', axis=1, inplace=True)
df['date'] = pd.to_datetime(df['date'],format='%Y-%m-%d') 
df = df[df.date<=datetime.strptime("2020-07-24", '%Y-%m-%d')]
display(df)

,date,state,city,place_type,confirmed,deaths,order_for_place,is_last,popEst,estimated_population,city_ibge_code,confirmed_per_100k_inhabitants,death_rate
217,2020-07-24,AP,TOTAL,state,35026,552,127,False,845731.0,861773.0,16.0,4064.41139,0.0158
218,2020-07-23,AP,TOTAL,state,34838,548,126,False,845731.0,861773.0,16.0,4042.59590,0.0157
219,2020-07-22,AP,TOTAL,state,34660,544,125,False,845731.0,861773.0,16.0,4021.94081,0.0157
220,2020-07-21,AP,TOTAL,state,34405,533,124,False,845731.0,861773.0,16.0,3992.35065,0.0155
221,2020-07-20,AP,TOTAL,state,34145,515,123,False,845731.0,861773.0,16.0,3962.18030,0.0151
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9475,2020-02-29,SP,TOTAL,state,2,0,5,False,45919049.0,46289333.0,35.0,0.00432,0.0000
9476,2020-02-28,SP,TOTAL,state,2,0,4,False,45919049.0,46289333.0,35.0,0.00432,0.0000
9477,2020-02-27,SP,TOTAL,state,1,0,3,False,45919049.0,46289333.0,35.0,0.00216,0.0000
9478,2020-02-26,SP,TOTAL,state,1,0,2,False,45919049.0,46289333.0,35.0,0.00216,0.0000


# Model

# SEAIR-D Model Equations

$$\Large \begin{array}{l}
\dfrac{d s}{d t}=-[ i(t) + \beta_R \cdot a(t)] \cdot \beta \cdot s(t) \\ 
\dfrac{d e}{d t}=[i(t) + \beta_R \cdot a(t)] \cdot \beta \cdot s(t) -\sigma \cdot e(t) - \mu \cdot e(t)\\ 
\dfrac{d a}{d t}=\sigma \cdot e(t) \cdot (1-p)-\gamma \cdot a(t)  \\
\dfrac{d i}{d t}=\sigma \cdot e(t) \cdot p - \gamma \cdot i(t)  - \mu \cdot i(t)\\ 
\dfrac{d r}{d t}=b_\gamma \cdot i(t) + \gamma \cdot a(t) + \nu \cdot k(t)- \mu \cdot r(t)\\
\dfrac{d k}{d t}=(a_\gamma + \nu ) \cdot k(t) + \mu \cdot [e(t)+i(t)+r(t)]
\end{array}$$

The recovery and death rates are calculated from $\gamma$

$$\Large \gamma=a_\gamma+b_\gamma$$

The sigmoid function:

$$\Large S(t)=\frac{1}{1 + e^{-t}}$$

The $\sigma$ variation is modeled as

$$\Large 
    \sigma = \sigma_i \cdot [1-S(t)]+\sigma_f \cdot S(t)
$$

See below the transition between two value - $\sigma_i$ and $\sigma_f$ -  based on Sigmoids Curves when $\sigma_i$<$\sigma_f$, which is indicated by the blue curve, and $\sigma_i$>$\sigma_f$, which is indicated by the red curve.

![Sigmoids Curves when $\sigma_i$<$\sigma_f$ and $\sigma_i$>$\sigma_f$](../countries/results/sigmoid.png)
<h4><center>Sigmoids Curves when $\sigma_i$<$\sigma_f$ and $\sigma_i$>$\sigma_f$</center></h4>


The $\sigma$ with sigmoid is performed two times inside the data interval, so we have an initial value constant of $\sigma_i$, a sigmoid transition to value $\sigma_f$, a period of $\sigma_f$ constant, after a sigmoid transition to value $\sigma_{ff}$ and a period of $\sigma_{ff}$ constant. The times where the two sigmoids are placed are calculated by optimization as well as the values of $\sigma_i$,$\sigma_f$ and $\sigma_{ff}$.

The sum of all people accounted is equal to population considered:

$$\Large s + e + a + i + r + k = N$$

The derivate is:

$$\Large \frac{d s}{d t}+\frac{d e}{d t}+\frac{d a}{d t}+\frac{d i}{d t}+\frac{d r}{d t}+\frac{d k}{d t} = 0$$

The last equation does not need to be solve because:

$$\Large \frac{d k}{d t}=-(\frac{d s}{d t}+\frac{d e}{d t}+\frac{d a}{d t}+\frac{d i}{d t}+\frac{d r}{d t})$$

The sum of all rates are equal to zero! The importance of this equation is that it conservates the rates.


## Parameters

### Fixed parameters:

N: the population considered in the study

$p$: is the fraction of the exposed which become symptomatic infectious sub-population.

$(1-p)$: is the fraction of the exposed which becomes asymptomatic infectious sub-population.

### Optimized Parameters

All parameters below are calculated through evolutionary optimization package Yabox to fit real data:

$\beta$: Effective contact rate [1/day]

$\beta_R$: Fraction of effective contact rate [1/day]
    
$\gamma$: Recovery(+Mortality) rate $\gamma=(a_\gamma+b_\gamma)$ [1/day]

$a_\gamma$: mortality of infected  [1/day]

$b_\gamma$: recovery rate  [1/day]

$\sigma$: is the rate at which individuals move from the exposed to the infectious classes. Three parameters $\sigma_i$, $\sigma_f$ and $\sigma_{ff}$ are used along the days. The transition from $\sigma_i$ to $\sigma_f$ and from $\sigma_f$ to $\sigma_{ff}$ are made by a sigmoid function. Its reciprocal  ($1/\sigma$) is the average latent (exposed) period. [1/day]

$\nu$: medicine and hospital improvement rate [1/day] 

$\mu$: direct mortality rate [1/day] 

# Initial parameters

In [5]:
paramOpt=4
changeCSV=False
adjustParam=False
paramSave=False
selectStates=False

## Select Parameters File, Version and Model

In [6]:
if paramOpt==0:
    paramFile="data/param.csv"
    version = "1"
    model = "ManualIC"

if paramOpt==1:
    paramFile="data/param_optimized_DE_Scipy.csv"    
    version = "1002"
    model = "DE_ScipyIC"
    
if paramOpt==2:
    paramFile="data/param_optimized_DE_SciPy_HistMin.csv"
    version = "1002"
    model = "DE_ScipyIC"

if paramOpt==3:
    paramFile="data/param_optimized_Yabox.csv"    
    version = "103"
    model = "YaboxIC"
    
if paramOpt==4:
    paramFile="data/param_optimized_Yabox_HistMin.csv"
    version = "105"
    model = "YaboxIC"

if paramOpt==5:
    paramFile="data/param_optimized_Brute_Force.csv"
    version = "12"
    model = "BruteForceIC"

if paramOpt==6:
    paramFile="data/param_optimized_Brute_Force_HistMin.csv"
    version = "12"
    model = "BruteForceIC"

## Final Adjust in Parameters

In [7]:
from datetime import datetime,timedelta,date

In [8]:
dfparam = pd.read_csv(paramFile)
dfparam = dfparam.dropna()

if selectStates:
    states=pd.Series(['SP','MG','AM','PE','ES','RJ'])
else:
    states=dfparam.state

if changeCSV:
    dfparam.loc[dfparam.state=='PI','d0'] = 0
    dfparam.loc[dfparam.state=='RR','d0'] = 0
    dfparam.loc[dfparam.state=='AP','d0'] = 0
    dfparam.loc[dfparam.state=='SP','WCASES'] = 0.65

if paramSave:
    dfparam.to_csv(paramFile)    
    
if adjustParam:
    sCorrect=[0.85,1.25,1.0,1.25,1.5,0.95,1.25,1.15,1.11,0.85,0.9]
    #103 [1.25,1.3,1.3,1.3,1.3,1.3,1.3,1.3,1.0,1.0,1.1]
    dfparam.s0=dfparam.s0.multiply(sCorrect[:len(states)], axis=0)

    

In [9]:
dfparam['WDTH']=1-dfparam['WCASES']-dfparam['WREC']

finalDate = date.today() + timedelta(days=-1)
finalDateStr = datetime.strftime(finalDate, "%Y-%m-%d")
dfparam['endDate']=finalDateStr

#poe RATIO no final
l=[0,1,2,3,4,5,6,7,8,9,11,12, 13, 14, 10] # index order
dfparam=dfparam[[dfparam.columns[i] for i in l]]    
    
dfparam

,state,start-date,prediction-range,s0,e0,a0,i0,r0,d0,START,WCASES,WREC,WDTH,endDate,RATIO
0,SP,2020-03-11,200,25252852,0,0,106,0,74,132,0.2730,0.0,0.7270,2021-03-02,0.15
1,ES,2020-04-02,200,1741035,0,0,51,0,172,127,0.2065,0.0,0.7935,2021-03-02,0.15
2,MG,2020-04-04,200,8771135,0,0,66,0,68,233,0.2471,0.0,0.7529,2021-03-02,0.15
3,RJ,2020-03-20,200,16171937,0,0,149,0,124,98,0.5386,0.0,0.4614,2021-03-02,0.15
4,CE,2020-03-17,200,7667224,0,0,67,0,112,33,0.4114,0.0,0.5886,2021-03-02,0.15
5,PE,2020-03-18,200,5591725,0,0,88,0,188,208,0.5481,0.0,0.4519,2021-03-02,0.15
6,BA,2020-03-17,200,12535116,0,0,230,0,202,19,0.2369,0.0,0.7631,2021-03-02,0.15
7,AM,2020-03-22,200,2110077,0,0,59,0,70,134,0.3215,0.0,0.6785,2021-03-02,0.15
8,PA,2020-03-18,200,4714263,0,0,186,0,73,75,0.4269,0.0,0.5731,2021-03-02,0.15
9,PI,2020-03-19,200,1611130,0,0,139,0,132,133,0.8110,0.0,0.1890,2021-03-02,0.15


# Define States Array and Import Solver

In [10]:
# Load solver
%reload_ext autoreload
%autoreload 2
import LearnerYabox_v3 as Learner #Yabox

2021-03-03 08:59:13,208	INFO services.py:1092 -- View the Ray dashboard at http://127.0.0.1:8266


## Activate Scrolling

In [11]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines){
    return true;}

<IPython.core.display.Javascript object>

## Select the Running Type

In [12]:
allStates=True
cleanRecovered=False
version="8"

display(states)

0     SP
1     ES
2     MG
3     RJ
4     CE
5     PE
6     BA
7     AM
8     PA
9     PI
10    RR
11    AP
12    PR
13    SC
14    RS
Name: state, dtype: object

## Main Loop

In [ ]:
# (self, state, start_date, predict_range,s_0, e_0, a_0, i_0, r_0, d_0, \
#     startNCases, weigthCases, weigthRecov, weigthDeath, end_date, ratio, cleanRecovered, version, \
#                  underNotif=True, Deaths=False, propWeigth=True, savedata=True):


results=[]
if allStates:
    
    loop=[0,6,12,17]
    for i in range(0,len(loop)-1):
        for state in states[loop[i]:loop[i+1]]:        
            query = dfparam.query('state == "{}"'.format(state)).reset_index()
            parameters = np.array(query.iloc[:, 2:])[0]
            f=Learner.Learner.remote(state, *parameters, cleanRecovered, version)
            result = f.train.remote() 
            results.append(result)
        display("Running States {:d} to {:d}".format(loop[i]+1,loop[i+1]))
        display(states[loop[i]:loop[i+1]])
        results = ray.get(results)
else:
    state= "RJ"
    query = dfparam.query('state == "{}"'.format(state)).reset_index()
    parameters = np.array(query.iloc[:, 2:])[0]
    parameters
    cleanRecovered = False
    f=Learner.Learner.remote(state, *parameters, cleanRecovered, version)
    result = f.train.remote() 
    results.append(result)
    results = ray.get(results)

# #execute all the queue with max_runner_cap at a time    


'Running States 1 to 6'

0    SP
1    ES
2    MG
3    RJ
4    CE
5    PE
Name: state, dtype: object

  0%|          | 0/10125000.0 [00:00<?, ?it/s]
  0%|          | 0/10125000.0 [00:00<?, ?it/s]
  0%|          | 0/10125000.0 [00:00<?, ?it/s]
  0%|          | 0/10125000.0 [00:00<?, ?it/s]
  0%|          | 0/10125000.0 [00:00<?, ?it/s]
  0%|          | 0/10125000.0 [00:00<?, ?it/s]
  0%|          | 0/10125000.0 [00:01<?, ?it/s]
  0%|          | 0/10125000.0 [00:01<?, ?it/s]
  0%|          | 0/10125000.0 [00:01<?, ?it/s]
  0%|          | 1/10125000.0 [00:02<1706:58:32,  1.65it/s]
  0%|          | 0/10125000.0 [00:01<?, ?it/s]
  0%|          | 1/10125000.0 [00:02<1680:28:51,  1.67it/s]
  0%|          | 0/10125000.0 [00:02<?, ?it/s]
  0%|          | 1/10125000.0 [00:02<2434:36:19,  1.16it/s]
  0%|          | 0/10125000.0 [00:02<?, ?it/s]
  0%|          | 3/10125000.0 [00:02<1448:11:58,  1.94it/s]
  0%|          | 1/10125000.0 [00:03<2818:27:00,  1.00s/it]
  0%|          | 3/10125000.0 [00:03<1686:47:26,  1.67it/s]
  0%|          | 1/10125000.0 [00:03<3345:39:30,  1.19s/it]
  0%|          |

  0%|          | 253/10125000.0 [00:17<106:18:02, 26.46it/s]
  0%|          | 231/10125000.0 [00:17<104:52:48, 26.82it/s]
  0%|          | 325/10125000.0 [00:17<79:05:06, 35.56it/s]
  0%|          | 231/10125000.0 [00:17<115:59:44, 24.25it/s]
  0%|          | 300/10125000.0 [00:18<91:11:47, 30.84it/s]
  0%|          | 231/10125000.0 [00:18<115:05:34, 24.44it/s]
  0%|          | 276/10125000.0 [00:18<99:23:41, 28.30it/s] 
  0%|          | 253/10125000.0 [00:18<98:10:12, 28.65it/s] 
  0%|          | 351/10125000.0 [00:18<75:58:25, 37.02it/s]
  0%|          | 253/10125000.0 [00:18<107:12:27, 26.23it/s]
  0%|          | 325/10125000.0 [00:19<87:10:09, 32.26it/s]
  0%|          | 300/10125000.0 [00:19<92:50:10, 30.29it/s]
  0%|          | 253/10125000.0 [00:19<108:19:46, 25.96it/s]
  0%|          | 276/10125000.0 [00:18<91:54:05, 30.60it/s]
  0%|          | 378/10125000.0 [00:19<72:56:56, 38.55it/s]
  0%|          | 325/10125000.0 [00:19<87:32:32, 32.13it/s]
  0%|          | 276/10125000.0 

  0%|          | 946/10125000.0 [00:35<54:18:06, 51.79it/s]
  0%|          | 903/10125000.0 [00:35<60:22:29, 46.58it/s]
  0%|          | 990/10125000.0 [00:35<56:23:15, 49.87it/s]
  0%|          | 1326/10125000.0 [00:35<47:17:50, 59.46it/s]
  0%|          | 1081/10125000.0 [00:36<62:26:13, 45.04it/s]
  0%|          | 1081/10125000.0 [00:36<60:11:31, 46.72it/s]
  0%|          | 990/10125000.0 [00:36<54:39:12, 51.46it/s]
  0%|          | 1035/10125000.0 [00:36<53:03:48, 53.00it/s]
  0%|          | 946/10125000.0 [00:36<59:36:39, 47.18it/s]
  0%|          | 1378/10125000.0 [00:36<44:56:53, 62.56it/s]
  0%|          | 1128/10125000.0 [00:37<57:08:52, 49.21it/s]
  0%|          | 1128/10125000.0 [00:37<54:39:32, 51.45it/s]
  0%|          | 1431/10125000.0 [00:37<41:10:05, 68.31it/s]
  0%|          | 1081/10125000.0 [00:37<50:28:39, 55.71it/s]
  0%|          | 990/10125000.0 [00:37<56:40:00, 49.63it/s]
  0%|          | 1035/10125000.0 [00:37<55:09:53, 50.98it/s]
  0%|          | 1176/10125000

...

  2%|▏         | 152076/10125000.0 [07:23<3:30:40, 788.94it/s]
  1%|          | 101475/10125000.0 [07:23<5:30:19, 505.74it/s]
  1%|          | 90100/10125000.0 [07:23<6:27:20, 431.79it/s]
  1%|          | 114960/10125000.0 [07:24<4:57:54, 560.02it/s]
  1%|          | 98790/10125000.0 [07:23<5:41:26, 489.40it/s]
  1%|          | 97461/10125000.0 [07:24<5:31:04, 504.81it/s]
  2%|▏         | 152628/10125000.0 [07:24<3:32:53, 780.70it/s]
  1%|          | 101926/10125000.0 [07:24<5:27:36, 509.92it/s]
  1%|          | 90525/10125000.0 [07:24<6:24:33, 434.89it/s]
  1%|          | 115440/10125000.0 [07:24<4:55:48, 563.98it/s]
  2%|▏         | 153181/10125000.0 [07:24<3:31:14, 786.78it/s]
  1%|          | 97903/10125000.0 [07:25<5:34:28, 499.64it/s]
  1%|          | 99235/10125000.0 [07:24<5:59:05, 465.34it/s]
  1%|          | 102378/10125000.0 [07:25<5:57:42, 466.98it/s]
  1%|          | 115921/10125000.0 [07:25<4:59:01, 557.87it/s]
  2%|▏         | 153735/10125000.0 [07:25<3:55:52, 704.54it/s

  1%|          | 125751/10125000.0 [07:43<4:50:18, 574.06it/s]
  2%|▏         | 167910/10125000.0 [07:43<3:12:46, 860.85it/s]
  1%|          | 106953/10125000.0 [07:44<5:29:46, 506.32it/s]
  1%|          | 107416/10125000.0 [07:43<6:19:46, 439.63it/s]
  1%|          | 98790/10125000.0 [07:44<6:20:55, 438.68it/s]
  1%|          | 111156/10125000.0 [07:44<5:43:34, 485.77it/s]
  1%|          | 126253/10125000.0 [07:44<4:44:40, 585.40it/s]
  2%|▏         | 168490/10125000.0 [07:44<3:11:40, 865.74it/s]
  1%|          | 107880/10125000.0 [07:44<6:06:45, 455.22it/s]
  2%|▏         | 169071/10125000.0 [07:45<3:11:45, 865.36it/s]
  1%|          | 107416/10125000.0 [07:45<5:58:03, 466.29it/s]
  1%|▏         | 126756/10125000.0 [07:45<4:42:10, 590.55it/s]
  1%|          | 99235/10125000.0 [07:45<6:41:59, 415.68it/s]
  1%|          | 111628/10125000.0 [07:45<6:35:43, 421.73it/s]
  2%|▏         | 169653/10125000.0 [07:45<3:11:24, 866.83it/s]
  1%|▏         | 127260/10125000.0 [07:46<4:47:08, 580.30

  2%|▏         | 182710/10125000.0 [08:05<6:07:26, 450.96it/s]
  1%|          | 120786/10125000.0 [08:05<5:02:31, 551.16it/s]
  1%|          | 107880/10125000.0 [08:05<6:07:41, 454.06it/s]
  1%|▏         | 138601/10125000.0 [08:05<4:23:56, 630.59it/s]
  1%|          | 117370/10125000.0 [08:05<5:08:51, 540.03it/s]
  2%|▏         | 183315/10125000.0 [08:05<5:10:26, 533.75it/s]
  1%|          | 117370/10125000.0 [08:05<5:30:43, 504.32it/s]
  1%|          | 121278/10125000.0 [08:05<5:00:54, 554.08it/s]
  1%|          | 108345/10125000.0 [08:06<6:00:48, 462.69it/s]
  2%|▏         | 183921/10125000.0 [08:06<4:30:19, 612.90it/s]
  1%|          | 117855/10125000.0 [08:06<5:05:18, 546.30it/s]
  1%|          | 117855/10125000.0 [08:06<5:30:09, 505.16it/s]
  1%|          | 121771/10125000.0 [08:06<4:52:16, 570.41it/s]
  2%|▏         | 184528/10125000.0 [08:06<4:03:18, 680.93it/s]
  1%|          | 108811/10125000.0 [08:07<5:57:02, 467.56it/s]
  1%|          | 118341/10125000.0 [08:07<5:02:09, 551.

  1%|          | 117855/10125000.0 [08:26<6:26:23, 431.64it/s]
  2%|▏         | 201295/10125000.0 [08:26<3:11:53, 861.92it/s]
  1%|▏         | 148785/10125000.0 [08:26<6:02:21, 458.86it/s]
  1%|▏         | 132870/10125000.0 [08:26<5:17:30, 524.51it/s]
  1%|▏         | 126756/10125000.0 [08:27<5:40:03, 490.03it/s]
  1%|          | 125751/10125000.0 [08:26<7:16:21, 381.93it/s]
  2%|▏         | 201930/10125000.0 [08:27<3:10:08, 869.83it/s]
  1%|▏         | 149331/10125000.0 [08:27<5:24:51, 511.80it/s]
  1%|          | 118341/10125000.0 [08:27<6:13:03, 447.06it/s]
  1%|▏         | 133386/10125000.0 [08:27<5:04:48, 546.33it/s]
  2%|▏         | 202566/10125000.0 [08:28<3:13:12, 855.94it/s]
  1%|▏         | 127260/10125000.0 [08:28<6:19:50, 438.68it/s]
  1%|          | 118828/10125000.0 [08:28<5:58:16, 465.48it/s]
  1%|          | 126253/10125000.0 [08:28<7:26:36, 373.14it/s]
  1%|▏         | 149878/10125000.0 [08:29<6:06:11, 454.00it/s]
  1%|▏         | 133903/10125000.0 [08:29<6:18:28, 439.

  2%|▏         | 219453/10125000.0 [08:47<3:19:46, 826.38it/s]
  2%|▏         | 162735/10125000.0 [08:47<3:50:51, 719.22it/s]
  1%|▏         | 142311/10125000.0 [08:47<4:43:44, 586.37it/s]
  1%|▏         | 137026/10125000.0 [08:48<4:59:20, 556.12it/s]
  2%|▏         | 220116/10125000.0 [08:48<3:23:22, 811.74it/s]
  2%|▏         | 163306/10125000.0 [08:48<3:56:00, 703.50it/s]
  1%|▏         | 135460/10125000.0 [08:48<5:37:22, 493.50it/s]
  1%|▏         | 128778/10125000.0 [08:48<6:05:11, 456.21it/s]
  1%|▏         | 142845/10125000.0 [08:48<5:17:32, 523.93it/s]
  2%|▏         | 220780/10125000.0 [08:49<3:19:59, 825.39it/s]
  1%|▏         | 137550/10125000.0 [08:49<5:13:42, 530.61it/s]
  2%|▏         | 163878/10125000.0 [08:49<3:54:37, 707.61it/s]
  1%|▏         | 129286/10125000.0 [08:49<5:50:41, 475.04it/s]
  2%|▏         | 221445/10125000.0 [08:49<3:10:54, 864.59it/s]
  1%|▏         | 135981/10125000.0 [08:49<6:14:32, 444.50it/s]
  2%|▏         | 164451/10125000.0 [08:50<3:49:16, 724.

In [ ]:
ray.shutdown()